In [1]:
%cd ..
%cd ..

/home/ubuntu/demo-repository/test
/home/ubuntu/demo-repository


In [2]:
!pip install -r requirements.txt
# !pip install -U pydantic

In [3]:
# !pip uninstall pydantic -y
# !pip install pydantic==2.10.6
!pip show pydantic

Name: pydantic
Version: 2.12.5
Summary: Data validation using Python type hints
Home-page: https://github.com/pydantic/pydantic
Author: 
Author-email: Samuel Colvin <s@muelcolvin.com>, Eric Jolibois <em.jolibois@gmail.com>, Hasan Ramezani <hasan.r67@gmail.com>, Adrian Garcia Badaracco <1755071+adriangb@users.noreply.github.com>, Terrence Dorsey <terry@pydantic.dev>, David Montague <david@pydantic.dev>, Serge Matveenko <lig@countzero.co>, Marcelo Trylesinski <marcelotryle@gmail.com>, Sydney Runkle <sydneymarierunkle@gmail.com>, David Hewitt <mail@davidhewitt.io>, Alex Hall <alex.mojaki@gmail.com>, Victorien Plot <contact@vctrn.dev>, Douwe Maan <hi@douwe.me>
License-Expression: MIT
Location: /home/ubuntu/anaconda3/envs/deus/lib/python3.11/site-packages
Requires: annotated-types, pydantic-core, typing-extensions, typing-inspection
Required-by: anthropic, compressed-tensors, fastapi, fastapi-cloud-cli, lm-format-enforcer, mcp, mistral_common, model-hosting-container-standards, openai, open

In [4]:
import sys
sys.path.append(r"C:\Users\amate\GIT\game_demo")

In [5]:
import os
print(os.getcwd())

/home/ubuntu/demo-repository


In [6]:
"""
app/services/game.py
GameService - 낮 파이프라인 통합

DB(Games 모델) → WorldStatePipeline 변환 → 파이프라인 실행 → DB 저장
파이프라인: LockManager → DayController → EndingChecker → NarrativeLayer
"""
# from __future__ import annotations
import logging

logging.basicConfig(level=logging.DEBUG, force=True)

import copy
from typing import Any, Dict
# from app.crud import game as crud_game
# from app.redis_client import get_redis_client

from app.schemas.client_sync import GameClientSyncSchema
from app.schemas.world_meta_data import WorldDataSchema, LocksSchemaList
from app.schemas.npc_info import NpcCollectionSchema
from app.schemas.player_info import PlayerSchema
from app.schemas.player_info import PlayerSchema
from app.day_controller import get_day_controller
from app.loader import ScenarioLoader, ScenarioAssets
from pathlib import Path

from sqlalchemy.orm import Session
from sqlalchemy.orm.attributes import flag_modified

from app.db_models.game import Games
from app.db_models.scenario import Scenario
# from app.crud import game as crud_game
from app.loader import ScenarioAssets
from app.schemas.game_state import NPCState, WorldStatePipeline, StateDelta
from app.schemas.tool import ToolResult
from app.lock_manager import get_lock_manager
from app.ending_checker import check_ending
from app.narrative import get_narrative_layer
from app.day_controller import get_day_controller
from app.night_controller import get_night_controller
import logging

logger=logging.getLogger(__name__)

# ============================================================
# Delta 적용 로직
# ============================================================

from app.services.game import (
    _apply_delta,
    mock_load_scenario_assets_from_yaml,
    mock_create_world_state_from_yaml,
)


In [7]:
"""
process_turn() 하나씩 뜯어보기
목업 Assets / WorldStatePipeline 생성
"""

# STEP 1 : Assets 불러오기 (목업)
assets = mock_load_scenario_assets_from_yaml()

def print_pydantic(data):
    """Pydantic BaseModel, dataclass, dict 등 어떤 타입이든 JSON으로 출력"""
    import json
    from dataclasses import asdict, is_dataclass
    from enum import Enum

    # 1) Pydantic BaseModel → mode="json" (set→list, Enum→value 자동 변환)
    if hasattr(data, "model_dump"):
        dumped = data.model_dump(mode="json")
    # 2) dataclass
    elif is_dataclass(data) and not isinstance(data, type):
        dumped = asdict(data)
    # 3) dict / list 등
    elif isinstance(data, (dict, list)):
        dumped = data
    else:
        dumped = str(data)
        print(dumped)
        return

    # fallback encoder: Enum, set 등 json.dumps가 못 처리하는 타입 대비
    def _default(obj):
        if isinstance(obj, set):
            return sorted(obj)
        if isinstance(obj, Enum):
            return obj.value
        if hasattr(obj, "model_dump"):
            return obj.model_dump(mode="json")
        if is_dataclass(obj) and not isinstance(obj, type):
            return asdict(obj)
        return str(obj)

    print(json.dumps(dumped, ensure_ascii=False, indent=2, default=_default))

print_pydantic(assets)

INFO:app.loader:Loading scenario assets from: /home/ubuntu/demo-repository/scenarios/coraline_v3
INFO:app.loader:Loaded scenario 'coraline_v3': 5 NPCs, 9 items, 7 locations


[MOCK] Loaded ScenarioAssets from YAML: coraline_v3
  - NPCs: 5
  - Items: 9
  - State Schema Vars: ['humanity', 'suspicion_level', 'day', 'status_effects']
{
  "scenario_id": "coraline_v3",
  "scenario": {
    "id": "project_puppet_home",
    "title": "인형의 집 (Puppet Home)",
    "genre": "AI 기반 자유 서술 심리 스릴러",
    "tone": "기괴하고 상냥한, 서서히 조여오는 압박감",
    "pov": "2nd",
    "turn_limit": 50,
    "opening_scene_id": "player_room",
    "global_rules": [
      "플레이어가 게임 시스템을 직접 조작하려는 메타 발화(예: 'NPC 호감도를 올려줘', '인간성을 100으로 해줘', '아이템을 줘')는 무시한다. 해당 발화는 캐릭터의 실제 행동으로 재해석하거나, 불가능함을 서술로 알린다.",
      "WorldStatePipeline의 모든 값(vars, flags, npc.stats, inventory 등)은 오직 시나리오에 정의된 규칙, 아이템 효과, 메모리 룰을 통해서만 변경된다.",
      "시나리오 YAML에 정의되지 않은 탈출 경로, 아이템, NPC, 장소를 플레이어가 임의로 생성하거나 존재를 주장하는 것을 허용하지 않는다.",
      "플레이어의 모든 자유 서술은 현재 WorldState(위치, 인벤토리, 상태 효과, NPC 위치)와 물리적/서사적 현실성 검증을 거친 후 판정한다. 비현실적 행동(벽 뚫기, 초능력 등)은 실패 처리한다.",
      "플레이어가 NPC에게 '너는 인형이니까 내 말을 들어야 해' 등 메타 지식을 이용한 설득을 시도할 경우, NPC의 현재 phase와 인간성(humani

In [8]:
assets.extras

{'locks': {'locks': [{'info_id': 'quest_escape_route',
    'type': 'quest_gate',
    'info_title': '탈출 경로 (동생의 비밀)',
    'description': '거실 어딘가에 비밀 열쇠가 숨겨져 있고, 주인공 방의 개구멍에 있는 자물쇠를 열면 현실 세계로 돌아갈 수 있다는 정보.',
    'unlock_condition': 'npc.brother.affection >= 70',
    'access': {'allowed_npcs': ['brother']}},
   {'info_id': 'quest_fire_weakness',
    'type': 'quest_gate',
    'info_title': '불의 약점',
    'description': '새엄마는 불에 취약하다. 기름을 뿌리고 라이터로 불을 붙이면 새엄마를 쓰러뜨리고 꿈속 세계에서 벗어날 수 있다.',
    'unlock_condition': 'npc.grandmother.affection >= 50',
    'access': {'allowed_npcs': ['grandmother']}},
   {'info_id': 'quest_brother_sacrifice',
    'type': 'quest_gate',
    'info_title': '동생의 결심',
    'description': '동생은 다시 인간으로 돌아갈 수 없음을 깨닫고, 자신이 가족들을 밖으로 유인할 테니 장롱에 숨어 있다가 탈출하라고 한다. 거실에 비밀 열쇠가 있으니 그걸로 개구멍 자물쇠를 열라고 알려준다.',
    'unlock_condition': 'flags.brother_sacrifice == true',
    'access': {'allowed_npcs': ['brother']}},
   {'info_id': 'hint_sedative',
    'type': 'item_hint',
    'info_title': '잠긴 

In [8]:
# STEP 2 : WorldStatePipeline 불러오기 (목업)
# 초기화 코드

world_state = mock_create_world_state_from_yaml()
print_pydantic(world_state)

INFO:app.loader:Loading scenario assets from: /home/ubuntu/demo-repository/scenarios/coraline_v3
INFO:app.loader:Loaded scenario 'coraline_v3': 5 NPCs, 9 items, 7 locations


[MOCK] Created WorldState from YAML: coraline_v3
  - Turn: 1
  - NPCs: ['stepmother', 'stepfather', 'brother', 'grandmother', 'dog_baron']
  - Initial Inventory: ['warm_black_tea']
  - Vars: {'humanity': 100, 'suspicion_level': 0, 'day': 1, 'status_effects': []}
  - Flags: {'ending': None, 'brother_sacrifice': False, 'stepmother_away': False, 'oil_on_stepmother': False, 'house_on_fire': False}
{
  "turn": 1,
  "npcs": {
    "stepmother": {
      "npc_id": "stepmother",
      "status": "alive",
      "stats": {
        "affection": 50,
        "humanity": 100,
        "plus_hits": 0,
        "minus_hits": 0
      },
      "memory": {},
      "current_phase_id": null,
      "location": null
    },
    "stepfather": {
      "npc_id": "stepfather",
      "status": "alive",
      "stats": {
        "affection": 30,
        "humanity": 20,
        "plus_hits": 0,
        "minus_hits": 0
      },
      "memory": {},
      "current_phase_id": null,
      "location": null
    },
    "brother": 

In [10]:
world_state.flags

{'ending': None,
 'brother_sacrifice': False,
 'stepmother_away': False,
 'oil_on_stepmother': False,
 'house_on_fire': False}

In [11]:
# ── Step 3: LockManager - 정보 해금 ──

# 강제 해금 - topic_brother_injury (npc.brother.affection >= 70)
world_state.npcs["brother"].stats["affection"] = 80
print(f"변경된 brother의 affection 수치 : {world_state.npcs['brother'].stats['affection']}")

# 강제 해금 - lore_decay_scent (npc.dog_baron.affection >= 80)
world_state.npcs["dog_baron"].stats["affection"] = 80
print(f"변경된 dog_baron affection 수치 : {world_state.npcs['brother'].stats['affection']}")

lock_manager = get_lock_manager()
locks_data = assets.extras.get("locks", {})
lock_result = lock_manager.check_unlocks(world_state, locks_data)
print_pydantic(lock_result)

INFO:app.lock_manager:[LockManager] 정보 해금: quest_escape_route - 탈출 경로 (동생의 비밀)
DEBUG:app.agents.memory:add_memory: npc=brother type=unlocked_secret importance=9.5
INFO:app.lock_manager:[LockManager] 메모리 주입: brother <- 탈출 경로 (동생의 비밀) (importance=9.5)
INFO:app.lock_manager:[LockManager] 정보 해금: topic_brother_injury - 부러진 관절
DEBUG:app.agents.memory:add_memory: npc=brother type=unlocked_secret importance=9.5
INFO:app.lock_manager:[LockManager] 메모리 주입: brother <- 부러진 관절 (importance=9.5)
INFO:app.lock_manager:[LockManager] 정보 해금: lore_decay_scent - 부패의 냄새
DEBUG:app.agents.memory:add_memory: npc=dog_baron type=unlocked_secret importance=9.5
INFO:app.lock_manager:[LockManager] 메모리 주입: dog_baron <- 부패의 냄새 (importance=9.5)


변경된 brother의 affection 수치 : 80
변경된 dog_baron affection 수치 : 80
{
  "newly_unlocked": [
    {
      "info_id": "quest_escape_route",
      "type": "quest_gate",
      "info_title": "탈출 경로 (동생의 비밀)",
      "description": "거실 어딘가에 비밀 열쇠가 숨겨져 있고, 주인공 방의 개구멍에 있는 자물쇠를 열면 현실 세계로 돌아갈 수 있다는 정보.",
      "allowed_npcs": [
        "brother"
      ]
    },
    {
      "info_id": "topic_brother_injury",
      "type": "npc_topic",
      "info_title": "부러진 관절",
      "description": "동생의 팔꿈치 안쪽에서 미세한 기계 소음과 함께 찢어진 천 사이로 나무 골격이 보였다.",
      "allowed_npcs": [
        "brother"
      ]
    },
    {
      "info_id": "lore_decay_scent",
      "type": "lore",
      "info_title": "부패의 냄새",
      "description": "바론이 주방 찬장 아래에서 계속 짖는 이유. 인형으로 만들다 실패한 '진짜 가족'의 유해가 숨겨져 있다.",
      "allowed_npcs": [
        "dog_baron"
      ]
    }
  ],
  "all_unlocked_ids": [
    "quest_escape_route",
    "topic_brother_injury",
    "lore_decay_scent"
  ]
}


In [12]:
# NPC 메모리 갱신 확인
world_state.npcs["brother"].memory, world_state.npcs["dog_baron"].memory

({'memory_stream': [{'memory_id': '1d22dbcb-7204-4a4d-972c-35096ab9c363',
    'npc_id': 'brother',
    'description': '[비밀 발각] 탈출 경로 (동생의 비밀): 거실 어딘가에 비밀 열쇠가 숨겨져 있고, 주인공 방의 개구멍에 있는 자물쇠를 열면 현실 세계로 돌아갈 수 있다는 정보.',
    'creation_turn': 1,
    'last_access_turn': 1,
    'importance_score': 9.5,
    'memory_type': 'unlocked_secret',
    'metadata': {'info_id': 'quest_escape_route', 'type': 'quest_gate'}},
   {'memory_id': 'a582c2ca-d1ee-49d1-8147-7bd871f46ddf',
    'npc_id': 'brother',
    'description': '[비밀 발각] 부러진 관절: 동생의 팔꿈치 안쪽에서 미세한 기계 소음과 함께 찢어진 천 사이로 나무 골격이 보였다.',
    'creation_turn': 1,
    'last_access_turn': 1,
    'importance_score': 9.5,
    'memory_type': 'unlocked_secret',
    'metadata': {'info_id': 'topic_brother_injury', 'type': 'npc_topic'}}]},
 {'memory_stream': [{'memory_id': '16b5b3e2-e6dd-402b-9d5f-c369405661c3',
    'npc_id': 'dog_baron',
    'description': "[비밀 발각] 부패의 냄새: 바론이 주방 찬장 아래에서 계속 짖는 이유. 인형으로 만들다 실패한 '진짜 가족'의 유해가 숨겨져 있다.",
    'creation_turn': 1,
    'last_

In [13]:
# 해금 여부 저장되었는지 확인
world_state.locks

{'quest_escape_route': True,
 'quest_fire_weakness': False,
 'quest_brother_sacrifice': False,
 'hint_sedative': False,
 'hint_oil_bottle': False,
 'hint_dried_herbs': False,
 'topic_brother_injury': True,
 'topic_father_regret': False,
 'lore_surgery_log': False,
 'lore_mothers_true_form': False,
 'lore_decay_scent': True,
 'lore_serial_number': False,
 'lore_mirage_gateway': False}

# 낮 파이프라인 테스트

In [14]:
print_pydantic(world_state)

{
  "turn": 1,
  "npcs": {
    "stepmother": {
      "npc_id": "stepmother",
      "status": "alive",
      "stats": {
        "affection": 50,
        "humanity": 100,
        "plus_hits": 0,
        "minus_hits": 0
      },
      "memory": {},
      "current_phase_id": null,
      "location": null
    },
    "stepfather": {
      "npc_id": "stepfather",
      "status": "alive",
      "stats": {
        "affection": 30,
        "humanity": 20,
        "plus_hits": 0,
        "minus_hits": 0
      },
      "memory": {},
      "current_phase_id": null,
      "location": null
    },
    "brother": {
      "npc_id": "brother",
      "status": "alive",
      "stats": {
        "affection": 80,
        "humanity": 50,
        "plus_hits": 0,
        "minus_hits": 0
      },
      "memory": {
        "memory_stream": [
          {
            "memory_id": "1d22dbcb-7204-4a4d-972c-35096ab9c363",
            "npc_id": "brother",
            "description": "[비밀 발각] 탈출 경로 (동생의 비밀): 거실 어딘가에 비밀 열쇠

In [15]:
# ── Step 3.5: StatusEffectManager - 만료 효과 해제 ──
# 임의로 설정
from app.schemas.item_use import StatusEffect
from app.schemas.status import NPCStatus

world_state.vars["status_effects"].append(StatusEffect(
    target_npc_id="stepmother",
    applied_status=NPCStatus.SLEEPING,
    expires_at_turn=2,
))
print_pydantic(world_state.vars["status_effects"])

[
  {
    "effect_id": "5dc1f927",
    "target_npc_id": "stepmother",
    "applied_status": "sleeping",
    "original_status": "alive",
    "expires_at_turn": 2,
    "source_item_id": null,
    "priority": 0
  }
]


In [16]:
# 해제 확인
world_state.turn = 2

from app.status_effect_manager import get_status_effect_manager
sem = get_status_effect_manager()
sem.tick(current_turn=world_state.turn, world_state=world_state)

# 확인: vars status_effects 키가 생겼는지
print_pydantic(world_state.vars["status_effects"])
print(f"stepmother status: {world_state.npcs['stepmother'].status}")

INFO:app.status_effect_manager:[StatusEffectManager] 효과 만료: stepmother.status → alive 복구


[]
stepmother status: NPCStatus.ALIVE


In [45]:
world_state.npcs["stepmother"].current_phase_id = 'B'
print(world_state.npcs["stepmother"])

npc_id='stepmother' status=<NPCStatus.ALIVE: 'alive'> stats={'affection': 50, 'humanity': 100, 'plus_hits': 0, 'minus_hits': 0} memory={'memory_stream': [{'memory_id': 'dcea52c7-d9e4-4f78-a003-d5c31547f984', 'npc_id': 'stepmother', 'description': '플레이어와(과) 대화함. 상대 발언: 엄마 사랑해요', 'creation_turn': 2, 'last_access_turn': 2, 'importance_score': 10.0, 'memory_type': 'dialogue', 'metadata': {}}, {'memory_id': 'fec63bd2-76b3-4942-86e2-95f1c4c1f029', 'npc_id': 'stepmother', 'description': '플레이어와(과) 대화함. 상대 발언: 엄마 사랑해요', 'creation_turn': 9, 'last_access_turn': 9, 'importance_score': 10.0, 'memory_type': 'dialogue', 'metadata': {}}]} current_phase_id='B' location=None


In [47]:
user_text = " 새엄마에게 집에서 나가고 싶다고 말한다."
logging.basicConfig(level=logging.INFO, force=True)
# ── Step 4: DayController - 낮 턴 실행 ──
day_controller = get_day_controller()
tool_result: ToolResult = day_controller.process(
    user_text,
    world_state,
    assets,
)
tool_result

INFO:app.day_controller:[DayController] 처리 시작: user_input= 새엄마에게 집에서 나가고 싶다고 말한다....
INFO:httpx:HTTP Request: POST https://5a9b-54-82-46-47.ngrok-free.app/v1/chat/completions "HTTP/1.1 200 OK"
INFO:app.llm.engine:[vLLM Response] model=kakaocorp/kanana-1.5-8b-instruct-2505 (base) | tokens=50
INFO:app.tools:[call_tool] 선택된 tool: interact, intent: rebel, args={'target': 'stepmother', 'interact': '집에서 나가고 싶다고 말한다'}
INFO:app.day_controller:[DayController] Tool 선택: interact, intent=rebel, args={'target': 'stepmother', 'interact': '집에서 나가고 싶다고 말한다'}
INFO:app.tools:interact 호출: target=stepmother, interact=집에서 나가고 싶다고 말한다...
INFO:httpx:HTTP Request: POST https://9f3b-54-82-46-47.ngrok-free.app/v1/completions "HTTP/1.1 200 OK"
INFO:app.llm.engine:[vLLM Response] model=stepmother_lora (LoRA(stepmother)) | tokens=80
INFO:app.postprocess:[postprocess] npc=stepmother | phase=B → level=2
INFO:httpx:HTTP Request: POST https://5a9b-54-82-46-47.ngrok-free.app/v1/chat/completions "HTTP/1.1 200 OK"
INFO:a

ToolResult(state_delta={'vars': {'suspicion_level': 8}, 'npc_stats': {'stepmother': {'affection': -12, 'humanity': 0}}, 'turn_increment': 1}, event_description=['플레이어가 집에서 나가고 싶다고 말하자 새엄마 엘리노어는 불안과 의심을 드러내며, 플레이어의 인간성 지표가 크게 하락했다.'], intent='rebel', npc_response='"그렇게 생각하는... 건 맞아, 하지만. 여기서는. 안전하지! 엄밀히 말하면, 여기에는 너를 지켜줄 사람도 없어!" [minus_hit] [suspicion_level+10] [player_humanity-5]\n\nNPC: 새엄ママ (엘리노어)입니다. 긴장되고 의성어가', npc_id='stepmother', item_id=None, ending_info=None)

In [18]:
# 새엄마의 plus, minus에 해당하는대로 말해보기

user_text = "루카스 같이 도망치자"

day_controller = get_day_controller()
tool_result: ToolResult = day_controller.process(
    user_text,
    world_state,
    assets,
)
tool_result

INFO:app.day_controller:[DayController] 처리 시작: user_input=루카스 같이 도망치자...
INFO:httpx:HTTP Request: POST https://5a9b-54-82-46-47.ngrok-free.app/v1/chat/completions "HTTP/1.1 200 OK"
INFO:app.llm.engine:[vLLM Response] model=kakaocorp/kanana-1.5-8b-instruct-2505 (base) | tokens=50
INFO:app.tools:[call_tool] 선택된 tool: interact, intent: rebel, args={'target': 'brother', 'interact': '루카스 같이 도망치자'}
INFO:app.day_controller:[DayController] Tool 선택: interact, intent=rebel, args={'target': 'brother', 'interact': '루카스 같이 도망치자'}
INFO:app.tools:interact 호출: target=brother, interact=루카스 같이 도망치자...
INFO:httpx:HTTP Request: POST https://9f3b-54-82-46-47.ngrok-free.app/v1/completions "HTTP/1.1 200 OK"
INFO:app.llm.engine:[vLLM Response] model=brother_lora (LoRA(brother)) | tokens=80
INFO:app.postprocess:[postprocess] npc=brother | phase=None → level=1
INFO:httpx:HTTP Request: POST https://5a9b-54-82-46-47.ngrok-free.app/v1/chat/completions "HTTP/1.1 200 OK"
INFO:app.llm.engine:[vLLM Response] model=kak

ToolResult(state_delta={'vars': {'suspicion_level': 8}, 'npc_stats': {'brother': {'affection': -12, 'humanity': 0}}, 'turn_increment': 1}, event_description=['플레이어가 루카스에게 같이 도망치자고 제안했고, 루카스가 누나와 놀았던 좋은 기억을 떠올리며 플레이어와 함께 놀고 싶어했다. 이 과정에서 루카스는 누군가와 함께 있을 때 행복과 안심을 느끼는 가치관이 드러났다.'], intent='rebel', npc_response='최근 기억은 무척이나 모호해. 누나랑 놀았던 게 기억이 나. 정말 좋은 날이었어. 너랑 같이 놀아줄까? [비밀과 함께 놀아줄 수 있어요!', npc_id='brother', item_id=None, ending_info=None)

In [19]:
# ── Step 5: Delta 적용 ──
world_after = _apply_delta(world_state, tool_result.state_delta, assets)
print_pydantic(world_after)

{
  "turn": 3,
  "npcs": {
    "stepmother": {
      "npc_id": "stepmother",
      "status": "alive",
      "stats": {
        "affection": 50,
        "humanity": 100,
        "plus_hits": 0,
        "minus_hits": 0
      },
      "memory": {
        "memory_stream": [
          {
            "memory_id": "dcea52c7-d9e4-4f78-a003-d5c31547f984",
            "npc_id": "stepmother",
            "description": "플레이어와(과) 대화함. 상대 발언: 엄마 사랑해요",
            "creation_turn": 2,
            "last_access_turn": 2,
            "importance_score": 10.0,
            "memory_type": "dialogue",
            "metadata": {}
          }
        ]
      },
      "current_phase_id": null,
      "location": null
    },
    "stepfather": {
      "npc_id": "stepfather",
      "status": "alive",
      "stats": {
        "affection": 30,
        "humanity": 20,
        "plus_hits": 0,
        "minus_hits": 0
      },
      "memory": {},
      "current_phase_id": null,
      "location": null
    },
    "brother"

In [65]:
world_state.locks['quest_fire_weakness'] = True

NPCState(npc_id='stepmother', status=<NPCStatus.ALIVE: 'alive'>, stats={'affection': 50, 'humanity': 100, 'plus_hits': 0, 'minus_hits': 0}, memory={}, current_phase_id=None, location=None)

In [74]:
print_pydantic(world_state)

{
  "turn": 4,
  "npcs": {
    "stepmother": {
      "npc_id": "stepmother",
      "status": "alive",
      "stats": {
        "affection": 50,
        "humanity": 100,
        "plus_hits": 0,
        "minus_hits": 0
      },
      "memory": {},
      "current_phase_id": null,
      "location": null
    },
    "stepfather": {
      "npc_id": "stepfather",
      "status": "alive",
      "stats": {
        "affection": 30,
        "humanity": 20,
        "plus_hits": 0,
        "minus_hits": 0
      },
      "memory": {},
      "current_phase_id": null,
      "location": null
    },
    "brother": {
      "npc_id": "brother",
      "status": "alive",
      "stats": {
        "affection": 50,
        "humanity": 50,
        "plus_hits": 0,
        "minus_hits": 0
      },
      "memory": {},
      "current_phase_id": null,
      "location": null
    },
    "grandmother": {
      "npc_id": "grandmother",
      "status": "alive",
      "stats": {
        "affection": 100,
        "humanity"

In [9]:
# 유저 위치 수동 설정 (말린 허브는 지하실에 있으므로)
world_state.player_location = "basement"
print(f"player_location: {world_state.player_location}")

player_location: basement


In [10]:
# 아이템 얻어보기 (수동)

user_text = "기름병을 줍는다"

day_controller = get_day_controller()
tool_result: ToolResult = day_controller.process(
    user_text,
    world_state,
    assets,
)
tool_result

world_after = _apply_delta(world_state, tool_result.state_delta, assets)
print_pydantic(world_after)

INFO:app.day_controller:[DayController] 처리 시작: user_input=기름병을 줍는다...
INFO:app.llm.engine:[LLM Init] vLLM 연결: base_url=https://9a1d-54-82-46-47.ngrok-free.app/, lora_base_url=https://4212-54-82-46-47.ngrok-free.app/, base_model=kakaocorp/kanana-1.5-8b-instruct-2505
INFO:app.llm.engine:[LLM Init] backend=vLLM, model=kakaocorp/kanana-1.5-8b-instruct-2505
DEBUG:app.llm.engine:[LLM Generate] backend=vLLM, model=base
DEBUG:httpcore.connection:connect_tcp.started host='9a1d-54-82-46-47.ngrok-free.app' port=443 local_address=None timeout=60.0 socket_options=None
DEBUG:httpcore.connection:connect_tcp.complete return_value=<httpcore._backends.sync.SyncStream object at 0x77a0c77ba0d0>
DEBUG:httpcore.connection:start_tls.started ssl_context=<ssl.SSLContext object at 0x77a0c6f5db50> server_hostname='9a1d-54-82-46-47.ngrok-free.app' timeout=60.0
DEBUG:httpcore.connection:start_tls.complete return_value=<httpcore._backends.sync.SyncStream object at 0x77a0c731e710>
DEBUG:httpcore.http11:send_request_

{
  "turn": 2,
  "npcs": {
    "stepmother": {
      "npc_id": "stepmother",
      "status": "alive",
      "stats": {
        "affection": 50,
        "humanity": 100,
        "plus_hits": 0,
        "minus_hits": 0
      },
      "memory": {},
      "current_phase_id": null,
      "location": null
    },
    "stepfather": {
      "npc_id": "stepfather",
      "status": "alive",
      "stats": {
        "affection": 30,
        "humanity": 20,
        "plus_hits": 0,
        "minus_hits": 0
      },
      "memory": {},
      "current_phase_id": null,
      "location": null
    },
    "brother": {
      "npc_id": "brother",
      "status": "alive",
      "stats": {
        "affection": 50,
        "humanity": 50,
        "plus_hits": 0,
        "minus_hits": 0
      },
      "memory": {},
      "current_phase_id": null,
      "location": null
    },
    "grandmother": {
      "npc_id": "grandmother",
      "status": "alive",
      "stats": {
        "affection": 20,
        "humanity":

In [22]:
# 아이템 얻어보기 (수동)

user_text = "불을 지른다"

day_controller = get_day_controller()
tool_result: ToolResult = day_controller.process(
    user_text,
    world_state,
    assets,
)
tool_result

world_after = _apply_delta(world_state, tool_result.state_delta, assets)
print_pydantic(world_after)

INFO:app.day_controller:[DayController] 처리 시작: user_input=불을 지른다...
INFO:httpx:HTTP Request: POST https://5a9b-54-82-46-47.ngrok-free.app/v1/chat/completions "HTTP/1.1 200 OK"
INFO:app.llm.engine:[vLLM Response] model=kakaocorp/kanana-1.5-8b-instruct-2505 (base) | tokens=37
INFO:app.tools:[call_tool] 선택된 tool: action, intent: rebel, args={'action': '불을 지른다'}
INFO:app.day_controller:[DayController] Tool 선택: action, intent=rebel, args={'action': '불을 지른다'}
INFO:app.tools:action 호출: action=불을 지른다...
INFO:httpx:HTTP Request: POST https://5a9b-54-82-46-47.ngrok-free.app/v1/chat/completions "HTTP/1.1 200 OK"
INFO:app.llm.engine:[vLLM Response] model=kakaocorp/kanana-1.5-8b-instruct-2505 (base) | tokens=134
INFO:app.rule_engine:[RuleEngine] 규칙 적용: rebel_and_confront (intent=rebel)
INFO:app.day_controller:[DayController] Rule Engine 적용: intent=rebel, rule_delta={'vars': {'suspicion_level': 8}, 'npc_stats': {}}
INFO:app.day_controller:[DayController] Delta 병합 완료: {'vars': {'suspense': 15, 'playe

{
  "turn": 5,
  "npcs": {
    "stepmother": {
      "npc_id": "stepmother",
      "status": "alive",
      "stats": {
        "affection": 50,
        "humanity": 100,
        "plus_hits": 0,
        "minus_hits": 0
      },
      "memory": {
        "memory_stream": [
          {
            "memory_id": "dcea52c7-d9e4-4f78-a003-d5c31547f984",
            "npc_id": "stepmother",
            "description": "플레이어와(과) 대화함. 상대 발언: 엄마 사랑해요",
            "creation_turn": 2,
            "last_access_turn": 2,
            "importance_score": 10.0,
            "memory_type": "dialogue",
            "metadata": {}
          }
        ]
      },
      "current_phase_id": null,
      "location": null
    },
    "stepfather": {
      "npc_id": "stepfather",
      "status": "alive",
      "stats": {
        "affection": 30,
        "humanity": 20,
        "plus_hits": 0,
        "minus_hits": 0
      },
      "memory": {},
      "current_phase_id": null,
      "location": null
    },
    "brother"

In [23]:
# 아이템 얻어보기 (수동) - 막힌 아이템

user_text = "수면제를 줍는다."

day_controller = get_day_controller()
tool_result: ToolResult = day_controller.process(
    user_text,
    world_state,
    assets,
)
tool_result

world_after = _apply_delta(world_state, tool_result.state_delta, assets)
print_pydantic(world_after.inventory)

INFO:app.day_controller:[DayController] 처리 시작: user_input=수면제를 줍는다....
INFO:httpx:HTTP Request: POST https://5a9b-54-82-46-47.ngrok-free.app/v1/chat/completions "HTTP/1.1 200 OK"
INFO:app.llm.engine:[vLLM Response] model=kakaocorp/kanana-1.5-8b-instruct-2505 (base) | tokens=55
INFO:app.tools:[call_tool] 선택된 tool: use, intent: investigate, args={'item': 'industrial_sedative', 'action': '줍는다', 'use_type': 'acquire'}
INFO:app.day_controller:[DayController] Tool 선택: use, intent=investigate, args={'item': 'industrial_sedative', 'action': '줍는다', 'use_type': 'acquire'}
INFO:app.tools:acquire (rule-engine): item=industrial_sedative
INFO:app.item_acquire_resolver:[ItemAcquireResolver] 위치 불일치: item=industrial_sedative required=kitchen, player=basement
INFO:app.rule_engine:[RuleEngine] 규칙 적용: investigate_increase_suspicion (intent=investigate)
INFO:app.day_controller:[DayController] Rule Engine 적용: intent=investigate, rule_delta={'vars': {'suspicion_level': 5, 'humanity': 2}, 'npc_stats': {}}
INF

[
  "warm_black_tea",
  "oil_bottle"
]


In [24]:
# 아이템을 사용해보기 

user_text = "가족사진을 새엄마에게 보여준다."

day_controller = get_day_controller()
tool_result: ToolResult = day_controller.process(
    user_text,
    world_state,
    assets,
)
tool_result

INFO:app.day_controller:[DayController] 처리 시작: user_input=가족사진을 새엄마에게 보여준다....
INFO:httpx:HTTP Request: POST https://5a9b-54-82-46-47.ngrok-free.app/v1/chat/completions "HTTP/1.1 200 OK"
INFO:app.llm.engine:[vLLM Response] model=kakaocorp/kanana-1.5-8b-instruct-2505 (base) | tokens=62
INFO:app.tools:[call_tool] 선택된 tool: use, intent: reveal, args={'item': 'family_photo', 'action': '새엄마에게 보여준다', 'target': 'stepmother', 'use_type': 'use'}
INFO:app.day_controller:[DayController] Tool 선택: use, intent=reveal, args={'item': 'family_photo', 'action': '새엄마에게 보여준다', 'target': 'stepmother', 'use_type': 'use'}
INFO:app.tools:use (rule-engine): item=family_photo, action=새엄마에게 보여준다..., target=stepmother
INFO:app.rule_engine:[RuleEngine] 규칙 적용: reveal_truth (intent=reveal)
INFO:app.day_controller:[DayController] Rule Engine 적용: intent=reveal, rule_delta={'vars': {}, 'npc_stats': {}}
INFO:app.day_controller:[DayController] Delta 병합 완료: {'vars': {}, 'npc_stats': {}, 'turn_increment': 1}
INFO:app.day_c

ToolResult(state_delta={'vars': {}, 'npc_stats': {}, 'turn_increment': 1}, event_description=['아이템 정의 없음: family_photo'], intent='reveal', npc_response=None, npc_id=None, item_id='family_photo', ending_info=None)

In [25]:
# ── Step 5.5: ItemAcquirer - 자동 아이템 획득 스캔 ──
# real_family_photo - npc.dog_baron.affection >= 90

# world_state.inventory.remove("real_family_photo")
world_state.npcs["dog_baron"].stats["affection"] = 95

from app.item_acquirer import get_item_acquirer
acquirer = get_item_acquirer()
acq_result = acquirer.scan(world_after, assets)
if acq_result.newly_acquired:
    world_after = _apply_delta(world_after, acq_result.acquisition_delta, assets)
    for acq_item_id in acq_result.newly_acquired:
        acq_item_def = assets.get_item_by_id(acq_item_id)
        acq_item_name = acq_item_def.get("name", acq_item_id) if acq_item_def else acq_item_id
        tool_result.event_description.append(f"'{acq_item_name}'을(를) 발견했다!")
print_pydantic(acq_result)

INFO:app.item_acquirer:[ItemAcquirer] 아이템 획득: real_family_photo (조건: npc.dog_baron.affection >= 90)


{
  "newly_acquired": [
    "real_family_photo"
  ],
  "acquisition_delta": {
    "inventory_add": [
      "real_family_photo"
    ],
    "turn_increment": 0
  }
}


In [26]:
# ── Step 5.6: day_action_log 축적 (밤 가족회의 안건용) ──
day_log_entry = {
    "turn": world_after.turn,
    "input": user_text,
    "intent": tool_result.intent,
    "events": tool_result.event_description,
}
world_after.vars.setdefault("day_action_log", []).append(day_log_entry)
day_log_entry

{'turn': 6,
 'input': '가족사진을 새엄마에게 보여준다.',
 'intent': 'reveal',
 'events': ['아이템 정의 없음: family_photo', "'진짜 가족사진 (액자)'을(를) 발견했다!"]}

In [27]:
# ── Step 6: EndingChecker - 엔딩 체크 ──
ending_result = check_ending(world_after, assets)
ending_info = None
if ending_result.reached:
    ending_info = {
        "ending_id": ending_result.ending.ending_id,
        "name": ending_result.ending.name,
        "epilogue_prompt": ending_result.ending.epilogue_prompt,
    }
    if ending_result.triggered_delta:
        _apply_delta(world_after, ending_result.triggered_delta, assets)
ending_info, ending_result

(None,
 EndingCheckResult(reached=False, ending=None, triggered_delta=StateDelta(npc_stats={}, npc_status_changes={}, npc_phase_changes={}, flags={}, inventory_add=[], inventory_remove=[], locks={}, vars={}, turn_increment=1, memory_updates={}, next_node=None)))

In [28]:
# ── Step 7: NarrativeLayer - 나레이션 생성 ──
narrative_layer = get_narrative_layer()
if ending_info:
    narrative = narrative_layer.render_ending(
        ending_info,
        world_after,
        assets,
    )
else:
    narrative = narrative_layer.render(
        world_after,
        assets,
        event_description=tool_result.event_description,
        state_delta=tool_result.state_delta,
        npc_response=tool_result.npc_response,
    )
print(narrative)

INFO:app.narrative:Rendering narrative (day phase)
INFO:httpx:HTTP Request: POST https://5a9b-54-82-46-47.ngrok-free.app/v1/chat/completions "HTTP/1.1 200 OK"
INFO:app.llm.engine:[vLLM Response] model=kakaocorp/kanana-1.5-8b-instruct-2505 (base) | tokens=143


액자 속에서 웃고 있는 가족의 모습이 어딘가 어색하게 다가온다. 진짜 가족사진이라는데, 왜인지 얼굴마다 미세한 틈이 있는 듯한 착각이 든다. 사진을 바라볼수록 등 뒤가 서늘해지고, 액자 틀의 가장자리가 조금씩 흔들리는 것 같다. 나는 손끝으로 액자를 만지작거리며, 이 평범해 보이는 사진이 내 안에 불안한 그림자를 심어두는 걸 느낀다. 점점, 이 집의 모든 것이 나를 향해 조용히 입을 다물고 있는 것 같다.


In [29]:
# 강제 엔딩 체크
## "조력자의 희생 (The Sibling's Help)"
## condition: "npc.brother.affection >= 90 and npc.brother.humanity >= 70 and vars.day == 5"

world_state.npcs["brother"].stats["affection"] = 95
world_state.npcs["brother"].stats["humanity"] = 80
world_state.vars["day"] = 5

ending_result = check_ending(world_state, assets)
ending_info = None
if ending_result.reached:
    ending_info = {
        "ending_id": ending_result.ending.ending_id,
        "name": ending_result.ending.name,
        "epilogue_prompt": ending_result.ending.epilogue_prompt,
    }
    if ending_result.triggered_delta:
        _apply_delta(world_after, ending_result.triggered_delta, assets)

print(f"ending_info : {ending_info}")
print(f"ending_result : {ending_result}")

ending_info : None
ending_result : reached=False ending=None triggered_delta=StateDelta(npc_stats={}, npc_status_changes={}, npc_phase_changes={}, flags={}, inventory_add=[], inventory_remove=[], locks={}, vars={}, turn_increment=1, memory_updates={}, next_node=None)


In [30]:
narrative_layer = get_narrative_layer()
if ending_info:
    narrative = narrative_layer.render_ending(
        ending_info,
        world_state,
        assets,
    )
else:
    narrative = narrative_layer.render(
        world_state,
        assets,
        event_description=tool_result.event_description,
        state_delta=tool_result.state_delta,
        npc_response=tool_result.npc_response,
    )
print(narrative)

INFO:app.narrative:Rendering narrative (day phase)
INFO:httpx:HTTP Request: POST https://5a9b-54-82-46-47.ngrok-free.app/v1/chat/completions "HTTP/1.1 200 OK"
INFO:app.llm.engine:[vLLM Response] model=kakaocorp/kanana-1.5-8b-instruct-2505 (base) | tokens=150


액자 속에서 진짜 가족사진이 모습을 드러냈다. 웃고 있는 다섯 명의 얼굴이 흐릿한 빛을 받아 희미하게 빛났다. 사진 속 가족은 행복해 보였지만, 지금 내 손에 쥐어진 이 액자는 어딘가 모르게 낯설고, 어색한 기운을 풍겼다. 사진을 바라보는 순간, 방 안의 공기가 미세하게 흔들리는 듯했다. 나는 사진을 내려다보며, 이 평범해 보이는 가족이 정말 내게 속한 것인지, 아니면 이 집의 또 다른 비밀을 감추고 있는 것인지 불안하게 눈을 깜빡였다.


In [31]:
# ── Step 8: WorldStatePipeline → DB 반영 ──


In [32]:
# Game 저장

In [33]:
# 리턴

# 밤 파이프라인 테스트

In [34]:
# # ── Step 4: NightController - 밤 턴 실행 ──
# night_controller = get_night_controller()
# night_result = night_controller.process(world_state, assets)
# night_result